In [1]:
from datetime import datetime
import tkinter as tk
from tkinter import messagebox
import cv2
import time
from PIL import Image, ImageTk
from my_functions import *

class App:
    def __init__(self, source):
        self.source = source
        self.cap = cv2.VideoCapture(source)
        self.frame_size = (640, 480)
        self.save_video = True
        self.show_video = True
        self.save_img = False
        self.notice_generated = False
        self.root = tk.Tk()
        self.root.title("Helmet Violation Detection")
        self.create_widgets()
        self.process_video()

    def create_widgets(self):
        self.label = tk.Label(self.root, text="Enter Vehicle Number:")
        self.label.pack()

        self.vehicle_number_entry = tk.Entry(self.root)
        self.vehicle_number_entry.pack()

        self.generate_button = tk.Button(self.root, text="Generate Notice", command=self.generate_notice)
        self.generate_button.pack()

    def generate_notice(self):
        vehicle_number = self.vehicle_number_entry.get()
        if vehicle_number.strip() == "":
            messagebox.showerror("Error", "Please enter the vehicle number.")
        else:
            notice = f"Notice: You were detected riding without a helmet and number plate on {datetime.now()}. Please ensure you wear a helmet for your safety and to comply with traffic regulations. Violation recorded with vehicle number: {vehicle_number}."
            messagebox.showinfo("Notice Generated", notice)
            self.notice_generated = True
            self.save_violation(vehicle_number)

    def save_violation(self, vehicle_number):
        ret, frame = self.cap.read()
        if ret:
            for _ in range(10):  # Skip some frames
                ret, frame = self.cap.read()
                if not ret:
                    break

            if ret:
                # Find the number plate and save it
                frame, results = object_detection(frame)
                number_plate_coords = find_number_plate(frame, results)
                if number_plate_coords:
                    x1, y1, x2, y2 = number_plate_coords
                    number_plate_img = frame[y1:y2, x1:x2]
                    cv2.imwrite(f'violations/{vehicle_number}_violation.jpg', number_plate_img)
                    print(f"Violation image saved for vehicle number {vehicle_number}")

    def process_video(self):
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter('output.avi', fourcc, 20.0, self.frame_size)

        while(self.cap.isOpened()):
            ret, frame = self.cap.read()
            if ret == True:
                frame = cv2.resize(frame, self.frame_size)
                original_frame = frame.copy()
                frame, results = object_detection(frame)

                if self.notice_generated:
                    # Convert frame to RGB for displaying in Tkinter
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    # Convert frame to ImageTk format
                    frame_tk = ImageTk.PhotoImage(image=Image.fromarray(frame_rgb))
                    # Display frame in Tkinter window
                    self.display_frame(frame_tk)

                rider_list = []
                head_list = []
                number_list = []

                for result in results:
                    x1,y1,x2,y2,cnf, clas = result
                    if clas == 0:
                        rider_list.append(result)
                    elif clas == 1:
                        head_list.append(result)
                    elif clas == 2:
                        number_list.append(result)

                for rdr in rider_list:
                    x1r, y1r, x2r, y2r, cnfr, clasr = rdr
                    for hd in head_list:
                        x1h, y1h, x2h, y2h, cnfh, clash = hd
                        if inside_box([x1r,y1r,x2r,y2r], [x1h,y1h,x2h,y2h]): 
                            try:
                                head_img = original_frame[y1h:y2h, x1h:x2h]
                                helmet_present = img_classify(head_img)
                            except:
                                helmet_present = [None]

                            if helmet_present[0] == False:
                                for num in number_list:
                                    x1_num, y1_num, x2_num, y2_num, conf_num, clas_num = num
                                    if inside_box([x1r,y1r,x2r,y2r], [x1_num, y1_num, x2_num, y2_num]):
                                        vehicle_number = extract_number_plate(original_frame, num)
                                        self.generate_notice(vehicle_number)
                                        break
            else:
                break

            if self.save_video:
                out.write(frame)
            if self.save_img:
                cv2.imwrite('saved_frame.jpg', frame)
            if self.show_video:
                cv2.imshow('Frame', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        self.cap.release()
        cv2.destroyAllWindows()
        print('Execution completed')

    def display_frame(self, frame):
        if hasattr(self, 'panel'):
            self.panel.configure(image=frame)
            self.panel.image = frame
        else:
            self.panel = tk.Label(self.root, image=frame)
            self.panel.pack()

if __name__ == "__main__":
    app = App('video.mp4')
    app.root.mainloop()


C:\Users\i\anaconda3\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Detected: head conf: 0.43  bbox: x1:436    y1:15    x2:536    y2:140
Detected: rider conf: 0.37  bbox: x1:414    y1:3    x2:576    y2:477
Detected: head conf: 0.62  bbox: x1:442    y1:311    x2:529    y2:414
Detected: rider conf: 0.56  bbox: x1:393    y1:301    x2:582    y2:477
Detected: head conf: 0.38  bbox: x1:397    y1:285    x2:574    y2:467
Detected: head conf: 0.65  bbox: x1:445    y1:200    x2:525    y2:306
Detected: rider conf: 0.59  bbox: x1:403    y1:191    x2:582    y2:478
Detected: rider conf: 0.66  bbox: x1:257    y1:334    x2:459    y2:480
Detected: head conf: 0.43  bbox: x1:307    y1:337    x2:385    y2:428
Detected: rider conf: 0.54  bbox: x1:235    y1:286    x2:460    y2:473
Detected: head conf: 0.4  bbox: x1:281    y1:303    x2:378    y2:389
Detected: rider conf: 0.56  bbox: x1:228    y1:261    x2:432    y2:479
Detected: rider conf: 0.46  bbox: x1:228    y1:233    x2:419    y2:469
Detected: rider conf: 0.37  bbox: x1:223    y1:183    x2:399    y2:475
Detected: rider 

In [1]:
import torch

print(torch.__version__)

2.1.0+cpu


In [21]:
import torch
from datetime import datetime
import tkinter as tk
from tkinter import messagebox
import cv2
import time
from PIL import Image, ImageTk
from my_functions import *



class App:
    def __init__(self, source):
        self.source = source
        self.cap = cv2.VideoCapture(source)
        self.frame_size = (640, 480)
        self.save_video = True
        self.show_video = True
        self.save_img = False
        self.notice_generated = False
        self.root = tk.Tk()
        self.root.title("Helmet Violation Detection")
        self.create_widgets()
        self.process_video()

    def create_widgets(self):
        self.label = tk.Label(self.root, text="Enter Vehicle Number:")
        self.label.pack()

        self.vehicle_number_entry = tk.Entry(self.root)
        self.vehicle_number_entry.pack()

        self.generate_button = tk.Button(self.root, text="Generate Notice", command=self.generate_notice)
        self.generate_button.pack()

    def generate_notice(self):
        vehicle_number = self.vehicle_number_entry.get()
        if vehicle_number.strip() == "":
            messagebox.showerror("Error", "Please enter the vehicle number.")
        else:
            notice = f"Notice: You were detected riding without a helmet and number plate on {datetime.now()}. Please ensure you wear a helmet for your safety and to comply with traffic regulations. Violation recorded with vehicle number: {vehicle_number}."
            messagebox.showinfo("Notice Generated", notice)
            self.notice_generated = True
            self.save_violation(vehicle_number)

    def save_violation(self, vehicle_number):
        ret, frame = self.cap.read()
        if ret:
            for _ in range(10):  # Skip some frames
                ret, frame = self.cap.read()
                if not ret:
                    break

            if ret:
                # Find the number plate and save it
                frame, results = object_detection(frame)
                number_plate_coords = find_number_plate(frame, results)
                if number_plate_coords:
                    x1, y1, x2, y2 = number_plate_coords
                    number_plate_img = frame[y1:y2, x1:x2]
                    cv2.imwrite(f'violations/{vehicle_number}_violation.jpg', number_plate_img)
                    print(f"Violation image saved for vehicle number {vehicle_number}")

    def process_video(self):
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter('output.avi', fourcc, 20.0, self.frame_size)

        while(self.cap.isOpened()):
            ret, frame = self.cap.read()
            if ret == True:
                frame = cv2.resize(frame, self.frame_size)
                original_frame = frame.copy()
                frame, results = object_detection(frame)

                if self.notice_generated:
                    # Convert frame to RGB for displaying in Tkinter
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    # Convert frame to ImageTk format
                    frame_tk = ImageTk.PhotoImage(image=Image.fromarray(frame_rgb))
                    # Display frame in Tkinter window
                    self.display_frame(frame_tk)

                rider_list = []
                head_list = []
                number_list = []

                for result in results:
                    x1,y1,x2,y2,cnf, clas = result
                    if clas == 0:
                        rider_list.append(result)
                    elif clas == 1:
                        head_list.append(result)
                    elif clas == 2:
                        number_list.append(result)

                for rdr in rider_list:
                    x1r, y1r, x2r, y2r, cnfr, clasr = rdr
                    for hd in head_list:
                        x1h, y1h, x2h, y2h, cnfh, clash = hd
                        if inside_box([x1r,y1r,x2r,y2r], [x1h,y1h,x2h,y2h]): 
                            try:
                                head_img = original_frame[y1h:y2h, x1h:x2h]
                                helmet_present = img_classify(head_img)
                            except:
                                helmet_present = [None]

                            if helmet_present[0] == False:
                                for num in number_list:
                                    x1_num, y1_num, x2_num, y2_num, conf_num, clas_num = num
                                    if inside_box([x1r,y1r,x2r,y2r], [x1_num, y1_num, x2_num, y2_num]):
                                        vehicle_number = extract_number_plate(original_frame, num)
                                        # Open the notice window and display frame image
                                        self.open_notice_window(vehicle_number, original_frame)
                                        break

            else:
                break

            if self.save_video:
                out.write(frame)
            if self.save_img:
                cv2.imwrite('saved_frame.jpg', frame)
            if self.show_video:
                cv2.imshow('Frame', frame)

            key = cv2.waitKey(1)
            if key & 0xFF == ord('q'):
                break

        self.cap.release()
        cv2.destroyAllWindows()
        print('Execution completed')

    def open_notice_window(self, vehicle_number, frame):
         # Generate notice
        notice = f"Notice: You were detected riding without a helmet and number plate on {datetime.now()}. Please ensure you wear a helmet for your safety and to comply with traffic regulations. Violation recorded with vehicle number: {vehicle_number}."
        messagebox.showinfo("Notice Generated", notice)

        # Display frame image
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_img = Image.fromarray(frame_rgb)
        frame_img.show()


    def display_frame(self, frame):
        if hasattr(self, 'panel'):
            self.panel.configure(image=frame)
            self.panel.image = frame
        else:
            self.panel = tk.Label(self.root, image=frame)
            self.panel.pack()

if __name__ == "__main__":
    app = App('video.mp4')
    app.root.mainloop()




Detected: head conf: 0.43  bbox: x1:436    y1:15    x2:536    y2:140
Detected: rider conf: 0.37  bbox: x1:414    y1:3    x2:576    y2:477
Detected: head conf: 0.62  bbox: x1:442    y1:311    x2:529    y2:414
Detected: rider conf: 0.56  bbox: x1:393    y1:301    x2:582    y2:477
Detected: head conf: 0.38  bbox: x1:397    y1:285    x2:574    y2:467
Detected: head conf: 0.65  bbox: x1:445    y1:200    x2:525    y2:306
Detected: rider conf: 0.59  bbox: x1:403    y1:191    x2:582    y2:478
Detected: rider conf: 0.66  bbox: x1:257    y1:334    x2:459    y2:480
Detected: head conf: 0.43  bbox: x1:307    y1:337    x2:385    y2:428
Detected: rider conf: 0.54  bbox: x1:235    y1:286    x2:460    y2:473
Detected: head conf: 0.4  bbox: x1:281    y1:303    x2:378    y2:389
Detected: rider conf: 0.56  bbox: x1:228    y1:261    x2:432    y2:479
Detected: rider conf: 0.46  bbox: x1:228    y1:233    x2:419    y2:469
Detected: rider conf: 0.37  bbox: x1:223    y1:183    x2:399    y2:475
Detected: rider 

In [20]:
def process_video(self):
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output.avi', fourcc, 20.0, self.frame_size)

    while(self.cap.isOpened()):
        ret, frame = self.cap.read()
        if ret == True:
            frame = cv2.resize(frame, self.frame_size)
            original_frame = frame.copy()
            frame, results = object_detection(frame)

            if self.notice_generated:
                # Convert frame to RGB for displaying in Tkinter
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # Convert frame to ImageTk format
                frame_tk = ImageTk.PhotoImage(image=Image.fromarray(frame_rgb))
                # Display frame in Tkinter window
                self.display_frame(frame_tk)

            rider_list = []
            head_list = []
            number_list = []

            for result in results:
                x1,y1,x2,y2,cnf, clas = result
                if clas == 0:
                    rider_list.append(result)
                elif clas == 1:
                    head_list.append(result)
                elif clas == 2:
                    number_list.append(result)

            for rdr in rider_list:
                x1r, y1r, x2r, y2r, cnfr, clasr = rdr
                for hd in head_list:
                    x1h, y1h, x2h, y2h, cnfh, clash = hd
                    if inside_box([x1r,y1r,x2r,y2r], [x1h,y1h,x2h,y2h]): 
                        try:
                            head_img = original_frame[y1h:y2h, x1h:x2h]
                            helmet_present = img_classify(head_img)
                        except:
                            helmet_present = [None]

                        if helmet_present[0] == False:
                            for num in number_list:
                                x1_num, y1_num, x2_num, y2_num, conf_num, clas_num = num
                                if inside_box([x1r,y1r,x2r,y2r], [x1_num, y1_num, x2_num, y2_num]):
                                    vehicle_number = extract_number_plate(original_frame, num)
                                    # Open the notice window and display frame image
                                    self.open_notice_window(vehicle_number, original_frame)
                                    break

        else:
            break

        if self.save_video:
            out.write(frame)
        if self.save_img:
            cv2.imwrite('saved_frame.jpg', frame)
        if self.show_video:
            cv2.imshow('Frame', frame)

        key = cv2.waitKey(1)
        if key & 0xFF == ord('q'):
            break

    self.cap.release()
    cv2.destroyAllWindows()
    print('Execution completed')

def open_notice_window(self, vehicle_number, frame):
    # Generate notice
    notice = f"Notice: You were detected riding without a helmet and number plate on {datetime.now()}. Please ensure you wear a helmet for your safety and to comply with traffic regulations. Violation recorded with vehicle number: {vehicle_number}."
    messagebox.showinfo("Notice Generated", notice)

    # Display frame image
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_img = Image.fromarray(frame_rgb)
    frame_img.show()


In [23]:
import cv2
from datetime import datetime
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
from my_functions import *

class App:
    def __init__(self, source):
        self.source = source
        self.cap = cv2.VideoCapture(source)
        self.frame_size = (640, 480)
        self.save_video = True
        self.show_video = True
        self.save_img = False
        self.notice_generated = False
        self.root = tk.Tk()
        self.root.title("Helmet Violation Detection")
        self.create_widgets()
        self.root.bind('n', self.generate_notice)  # Bind 'n' key to generate_notice method
        self.process_video()

    def create_widgets(self):
        self.label = tk.Label(self.root, text="Enter Vehicle Number:")
        self.label.pack()

        self.vehicle_number_entry = tk.Entry(self.root)
        self.vehicle_number_entry.pack()
        
      
    def generate_notice(self, event=None):  # Accept event parameter for binding
        vehicle_number = self.vehicle_number_entry.get()
        if vehicle_number.strip() == "":
            messagebox.showerror("Error", "Please enter the vehicle number.")
        else:
            # Display the frame image
            ret, frame = self.cap.read()
            if ret:
                frame = cv2.resize(frame, self.frame_size)
                original_frame = frame.copy()
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_img = Image.fromarray(frame_rgb)
                frame_img.show()

            # Generate the notice
            notice = f"Notice: You were detected riding without a helmet and number plate on {datetime.now()}. Please ensure you wear a helmet for your safety and to comply with traffic regulations. Violation recorded with vehicle number: {vehicle_number}."
            messagebox.showinfo("Notice Generated", notice)
            self.notice_generated = True
            self.save_violation(vehicle_number)
        ##############################################################

    #def generate_notice(self, event=None):  # Accept event parameter for binding
      #  vehicle_number = self.vehicle_number_entry.get()
       # if vehicle_number.strip() == "":
        #    messagebox.showerror("Error", "Please enter the vehicle number.")
        #else:
         #   notice = f"Notice: You were detected riding without a helmet and number plate on {datetime.now()}. Please ensure you wear a helmet for your safety and to comply with traffic regulations. Violation recorded with vehicle number: {vehicle_number}."
          #  messagebox.showinfo("Notice Generated", notice)
           # self.notice_generated = True
            #self.save_violation(vehicle_number)
            ################################################

    def save_violation(self, vehicle_number):
        ret, frame = self.cap.read()
        if ret:
            for _ in range(10):  # Skip some frames
                ret, frame = self.cap.read()
                if not ret:
                    break

            if ret:
                # Find the number plate and save it
                frame, results = object_detection(frame)
                number_plate_coords = find_number_plate(frame, results)
                if number_plate_coords:
                    x1, y1, x2, y2 = number_plate_coords
                    number_plate_img = frame[y1:y2, x1:x2]
                    cv2.imwrite(f'violations/{vehicle_number}_violation.jpg', number_plate_img)
                    print(f"Violation image saved for vehicle number {vehicle_number}")

    def process_video(self):
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter('output.avi', fourcc, 20.0, self.frame_size)

        while(self.cap.isOpened()):
            ret, frame = self.cap.read()
            if ret == True:
                frame = cv2.resize(frame, self.frame_size)
                original_frame = frame.copy()
                frame, results = object_detection(frame)

                if self.notice_generated:
                    # Convert frame to RGB for displaying in Tkinter
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    # Convert frame to ImageTk format
                    frame_tk = ImageTk.PhotoImage(image=Image.fromarray(frame_rgb))
                    # Display frame in Tkinter window
                    self.display_frame(frame_tk)

                rider_list = []
                head_list = []
                number_list = []

                for result in results:
                    x1,y1,x2,y2,cnf, clas = result
                    if clas == 0:
                        rider_list.append(result)
                    elif clas == 1:
                        head_list.append(result)
                    elif clas == 2:
                        number_list.append(result)

                for rdr in rider_list:
                    x1r, y1r, x2r, y2r, cnfr, clasr = rdr
                    for hd in head_list:
                        x1h, y1h, x2h, y2h, cnfh, clash = hd
                        if inside_box([x1r,y1r,x2r,y2r], [x1h,y1h,x2h,y2h]): 
                            try:
                                head_img = original_frame[y1h:y2h, x1h:x2h]
                                helmet_present = img_classify(head_img)
                            except:
                                helmet_present = [None]

                            if helmet_present[0] == False:
                                for num in number_list:
                                    x1_num, y1_num, x2_num, y2_num, conf_num, clas_num = num
                                    if inside_box([x1r,y1r,x2r,y2r], [x1_num, y1_num, x2_num, y2_num]):
                                         vehicle_number = extract_number_plate(original_frame, num)
                                         self.generate_notice(vehicle_number)
                                         break
            else:
                break

            if self.save_video:
                out.write(frame)
            if self.save_img:
                cv2.imwrite('saved_frame.jpg', frame)
            if self.show_video:
                cv2.imshow('Frame', frame)

            key = cv2.waitKey(1)
            if key & 0xFF == ord('q'):
                break

        self.cap.release()
        cv2.destroyAllWindows()
        print('Execution completed')

    def display_frame(self, frame):
        if hasattr(self, 'panel'):
            self.panel.configure(image=frame)
            self.panel.image = frame
        else:
            self.panel = tk.Label(self.root, image=frame)
            self.panel.pack()

if __name__ == "__main__":
    app = App('video.mp4')
    app.root.mainloop()


Detected: head conf: 0.43  bbox: x1:436    y1:15    x2:536    y2:140
Detected: rider conf: 0.37  bbox: x1:414    y1:3    x2:576    y2:477
Detected: head conf: 0.62  bbox: x1:442    y1:311    x2:529    y2:414
Detected: rider conf: 0.56  bbox: x1:393    y1:301    x2:582    y2:477
Detected: head conf: 0.38  bbox: x1:397    y1:285    x2:574    y2:467
Detected: head conf: 0.65  bbox: x1:445    y1:200    x2:525    y2:306
Detected: rider conf: 0.59  bbox: x1:403    y1:191    x2:582    y2:478
Detected: rider conf: 0.66  bbox: x1:257    y1:334    x2:459    y2:480
Detected: head conf: 0.43  bbox: x1:307    y1:337    x2:385    y2:428
Detected: rider conf: 0.54  bbox: x1:235    y1:286    x2:460    y2:473
Detected: head conf: 0.4  bbox: x1:281    y1:303    x2:378    y2:389
Detected: rider conf: 0.56  bbox: x1:228    y1:261    x2:432    y2:479
Detected: rider conf: 0.46  bbox: x1:228    y1:233    x2:419    y2:469
Detected: rider conf: 0.37  bbox: x1:223    y1:183    x2:399    y2:475
Detected: rider 

In [25]:
import cv2
from datetime import datetime
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
from my_functions import *

class App:
    def __init__(self, source):
        self.source = source
        self.cap = cv2.VideoCapture(source)
        self.frame_size = (640, 480)
        self.save_video = True
        self.show_video = True
        self.save_img = False
        self.notice_generated = False
        self.root = tk.Tk()
        self.root.title("Helmet Violation Detection")
        self.create_widgets()
        self.root.bind('n', self.handle_n_key)  # Bind 'n' key to handle_n_key method
        self.process_video()

    def create_widgets(self):
        self.label = tk.Label(self.root, text="Enter Vehicle Number:")
        self.label.pack()

        self.vehicle_number_entry = tk.Entry(self.root)
        self.vehicle_number_entry.pack()
        
        self.generate_button = tk.Button(self.root, text="Generate Notice", command=self.generate_notice)
        self.generate_button.pack()


    def generate_notice(self, vehicle_number):
        notice = f"Notice: You were detected riding without a helmet and number plate on {datetime.now()}. Please ensure you wear a helmet for your safety and to comply with traffic regulations. Violation recorded with vehicle number: {vehicle_number}."
        messagebox.showinfo("Notice Generated", notice)

    def save_violation(self, vehicle_number):
        ret, frame = self.cap.read()
        if ret:
            for _ in range(10):  # Skip some frames
                ret, frame = self.cap.read()
                if not ret:
                    break

            if ret:
                # Find the number plate and save it
                frame, results = object_detection(frame)
                number_plate_coords = find_number_plate(frame, results)
                if number_plate_coords:
                    x1, y1, x2, y2 = number_plate_coords
                    number_plate_img = frame[y1:y2, x1:x2]
                    cv2.imwrite(f'violations/{vehicle_number}_violation.jpg', number_plate_img)
                    print(f"Violation image saved for vehicle number {vehicle_number}")

    def handle_n_key(self, event):
        vehicle_number = self.vehicle_number_entry.get()
        if vehicle_number.strip() == "":
            messagebox.showerror("Error", "Please enter the vehicle number.")
        else:
            ret, frame = self.cap.read()
            if ret:
                frame = cv2.resize(frame, self.frame_size)
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_img = Image.fromarray(frame_rgb)
                frame_img.show()
                self.generate_notice(vehicle_number)
                self.save_violation(vehicle_number)

    def process_video(self):
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter('output.avi', fourcc, 20.0, self.frame_size)

        while self.cap.isOpened():
            ret, frame = self.cap.read()
            if ret == True:
                frame = cv2.resize(frame, self.frame_size)
                original_frame = frame.copy()
                frame, results = object_detection(frame)

                if self.notice_generated:
                    # Convert frame to RGB for displaying in Tkinter
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    # Convert frame to ImageTk format
                    frame_tk = ImageTk.PhotoImage(image=Image.fromarray(frame_rgb))
                    # Display frame in Tkinter window
                    self.display_frame(frame_tk)

                rider_list = []
                head_list = []
                number_list = []

                for result in results:
                    x1,y1,x2,y2,cnf, clas = result
                    if clas == 0:
                        rider_list.append(result)
                    elif clas == 1:
                        head_list.append(result)
                    elif clas == 2:
                        number_list.append(result)

                for rdr in rider_list:
                    x1r, y1r, x2r, y2r, cnfr, clasr = rdr
                    for hd in head_list:
                        x1h, y1h, x2h, y2h, cnfh, clash = hd
                        if inside_box([x1r,y1r,x2r,y2r], [x1h,y1h,x2h,y2h]): 
                            try:
                                head_img = original_frame[y1h:y2h, x1h:x2h]
                                helmet_present = img_classify(head_img)
                            except:
                                helmet_present = [None]

                            if helmet_present[0] == False:
                                for num in number_list:
                                    x1_num, y1_num, x2_num, y2_num, conf_num, clas_num = num
                                    if inside_box([x1r,y1r,x2r,y2r], [x1_num, y1_num, x2_num, y2_num]):
                                        vehicle_number = extract_number_plate(original_frame, num)
                                        self.generate_notice(vehicle_number)
                                        break
            else:
                break

            if self.save_video:
                out.write(frame)
            if self.save_img:
                cv2.imwrite('saved_frame.jpg', frame)
            if self.show_video:
                cv2.imshow('Frame', frame)

            key = cv2.waitKey(1)
            if key & 0xFF == ord('q'):
                break

        self.cap.release()
        cv2.destroyAllWindows()
        print('Execution completed')

    def display_frame(self, frame):
        if hasattr(self, 'panel'):
            self.panel.configure(image=frame)
            self.panel.image = frame
        else:
            self.panel = tk.Label(self.root, image=frame)
            self.panel.pack()

if __name__ == "__main__":
    app = App('video.mp4')
    app.root.mainloop()


Detected: head conf: 0.43  bbox: x1:436    y1:15    x2:536    y2:140
Detected: rider conf: 0.37  bbox: x1:414    y1:3    x2:576    y2:477
Detected: head conf: 0.62  bbox: x1:442    y1:311    x2:529    y2:414
Detected: rider conf: 0.56  bbox: x1:393    y1:301    x2:582    y2:477
Detected: head conf: 0.38  bbox: x1:397    y1:285    x2:574    y2:467
Detected: head conf: 0.65  bbox: x1:445    y1:200    x2:525    y2:306
Detected: rider conf: 0.59  bbox: x1:403    y1:191    x2:582    y2:478
Detected: rider conf: 0.66  bbox: x1:257    y1:334    x2:459    y2:480
Detected: head conf: 0.43  bbox: x1:307    y1:337    x2:385    y2:428
Detected: rider conf: 0.54  bbox: x1:235    y1:286    x2:460    y2:473
Detected: head conf: 0.4  bbox: x1:281    y1:303    x2:378    y2:389
Detected: rider conf: 0.56  bbox: x1:228    y1:261    x2:432    y2:479
Detected: rider conf: 0.46  bbox: x1:228    y1:233    x2:419    y2:469
Detected: rider conf: 0.37  bbox: x1:223    y1:183    x2:399    y2:475
Detected: rider 

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\i\anaconda3\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
TypeError: App.generate_notice() missing 1 required positional argument: 'vehicle_number'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\i\anaconda3\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
TypeError: App.generate_notice() missing 1 required positional argument: 'vehicle_number'


In [27]:
import cv2
from datetime import datetime
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
from my_functions import *

class App:
    def __init__(self, source):
        self.source = source
        self.cap = cv2.VideoCapture(source)
        self.frame_size = (640, 480)
        self.save_video = True
        self.show_video = True
        self.save_img = False
        self.notice_generated = False
        self.root = tk.Tk()
        self.root.title("Helmet Violation Detection")
        self.create_widgets()
        self.process_video()

    def create_widgets(self):
        self.label = tk.Label(self.root, text="Enter Vehicle Number:")
        self.label.pack()

        self.vehicle_number_entry = tk.Entry(self.root)
        self.vehicle_number_entry.pack()

    def generate_notice(self, vehicle_number):
        notice = f"Notice: You were detected riding without a helmet and number plate on {datetime.now()}. Please ensure you wear a helmet for your safety and to comply with traffic regulations. Violation recorded with vehicle number: {vehicle_number}."
        messagebox.showinfo("Notice Generated", notice)

    def save_violation(self, vehicle_number):
        ret, frame = self.cap.read()
        if ret:
            for _ in range(10):  # Skip some frames
                ret, frame = self.cap.read()
                if not ret:
                    break

            if ret:
                # Find the number plate and save it
                frame, results = object_detection(frame)
                number_plate_coords = find_number_plate(frame, results)
                if number_plate_coords:
                    x1, y1, x2, y2 = number_plate_coords
                    number_plate_img = frame[y1:y2, x1:x2]
                    cv2.imwrite(f'violations/{vehicle_number}_violation.jpg', number_plate_img)
                    print(f"Violation image saved for vehicle number {vehicle_number}")

    def process_video(self):
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter('output.avi', fourcc, 20.0, self.frame_size)

        while(self.cap.isOpened()):
            ret, frame = self.cap.read()
            if ret == True:
                frame = cv2.resize(frame, self.frame_size)
                original_frame = frame.copy()
                frame, results = object_detection(frame)

                if self.notice_generated:
                    # Convert frame to RGB for displaying in Tkinter
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    # Convert frame to ImageTk format
                    frame_tk = ImageTk.PhotoImage(image=Image.fromarray(frame_rgb))
                    # Display frame in Tkinter window
                    self.display_frame(frame_tk)

                rider_list = []
                head_list = []
                number_list = []

                for result in results:
                    x1,y1,x2,y2,cnf, clas = result
                    if clas == 0:
                        rider_list.append(result)
                    elif clas == 1:
                        head_list.append(result)
                    elif clas == 2:
                        number_list.append(result)

                for rdr in rider_list:
                    x1r, y1r, x2r, y2r, cnfr, clasr = rdr
                    for hd in head_list:
                        x1h, y1h, x2h, y2h, cnfh, clash = hd
                        if inside_box([x1r,y1r,x2r,y2r], [x1h,y1h,x2h,y2h]): 
                            try:
                                head_img = original_frame[y1h:y2h, x1h:x2h]
                                helmet_present = img_classify(head_img)
                            except:
                                helmet_present = [None]

                            if helmet_present[0] == False:
                                for num in number_list:
                                    x1_num, y1_num, x2_num, y2_num, conf_num, clas_num = num
                                    if inside_box([x1r,y1r,x2r,y2r], [x1_num, y1_num, x2_num, y2_num]):
                                        vehicle_number = extract_number_plate(original_frame, num)
                                        self.notice_generated = True
                                        self.save_violation(vehicle_number)
                                        self.generate_notice(vehicle_number)
                                        break
            else:
                break

            if self.save_video:
                out.write(frame)
            if self.save_img:
                cv2.imwrite('saved_frame.jpg', frame)
            if self.show_video:
                cv2.imshow('Frame', frame)

            key = cv2.waitKey(1)
            if key & 0xFF == ord('q'):
                break
            elif key & 0xFF == ord('n'):
                vehicle_number = self.vehicle_number_entry.get()
                if vehicle_number.strip() != "":
                    self.notice_generated = False
                    self.generate_notice(vehicle_number)

        self.cap.release()
        cv2.destroyAllWindows()
        print('Execution completed')

    def display_frame(self, frame):
        if hasattr(self, 'panel'):
            self.panel.configure(image=frame)
            self.panel.image = frame
        else:
            self.panel = tk.Label(self.root, image=frame)
            self.panel.pack()

if __name__ == "__main__":
    app = App('video.mp4')
    app.root.mainloop()


Detected: head conf: 0.43  bbox: x1:436    y1:15    x2:536    y2:140
Detected: rider conf: 0.37  bbox: x1:414    y1:3    x2:576    y2:477
Detected: head conf: 0.62  bbox: x1:442    y1:311    x2:529    y2:414
Detected: rider conf: 0.56  bbox: x1:393    y1:301    x2:582    y2:477
Detected: head conf: 0.38  bbox: x1:397    y1:285    x2:574    y2:467
Detected: head conf: 0.65  bbox: x1:445    y1:200    x2:525    y2:306
Detected: rider conf: 0.59  bbox: x1:403    y1:191    x2:582    y2:478
Detected: rider conf: 0.66  bbox: x1:257    y1:334    x2:459    y2:480
Detected: head conf: 0.43  bbox: x1:307    y1:337    x2:385    y2:428
Detected: rider conf: 0.54  bbox: x1:235    y1:286    x2:460    y2:473
Detected: head conf: 0.4  bbox: x1:281    y1:303    x2:378    y2:389
Detected: rider conf: 0.56  bbox: x1:228    y1:261    x2:432    y2:479
Detected: rider conf: 0.46  bbox: x1:228    y1:233    x2:419    y2:469
Detected: rider conf: 0.37  bbox: x1:223    y1:183    x2:399    y2:475
Detected: rider 

In [28]:
import cv2
from datetime import datetime
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
from my_functions import *

class App:
    def __init__(self, source):
        self.source = source
        self.cap = cv2.VideoCapture(source)
        self.frame_size = (640, 480)
        self.save_video = True
        self.show_video = True
        self.save_img = False
        self.notice_generated = False
        self.root = tk.Tk()
        self.root.title("Helmet Violation Detection")
        self.create_widgets()
        self.process_video()

    def create_widgets(self):
        self.label = tk.Label(self.root, text="Enter Vehicle Number:")
        self.label.pack()

        self.vehicle_number_entry = tk.Entry(self.root)
        self.vehicle_number_entry.pack()

        # Bind the 'n' key to generate_notice method
        self.root.bind('n', self.generate_notice)

    def generate_notice(self, event=None):
        vehicle_number = self.vehicle_number_entry.get()
        if vehicle_number.strip() == "":
            messagebox.showerror("Error", "Please enter the vehicle number.")
        else:
            ret, frame = self.cap.read()
            if ret:
                frame = cv2.resize(frame, self.frame_size)
                original_frame = frame.copy()
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_img = Image.fromarray(frame_rgb)
                frame_img.show()
                notice = f"Notice: You were detected riding without a helmet and number plate on {datetime.now()}. Please ensure you wear a helmet for your safety and to comply with traffic regulations. Violation recorded with vehicle number: {vehicle_number}."
                messagebox.showinfo("Notice Generated", notice)
                self.notice_generated = True
                self.save_violation(vehicle_number)

    def save_violation(self, vehicle_number):
        ret, frame = self.cap.read()
        if ret:
            for _ in range(10):  # Skip some frames
                ret, frame = self.cap.read()
                if not ret:
                    break

            if ret:
                # Find the number plate and save it
                frame, results = object_detection(frame)
                number_plate_coords = find_number_plate(frame, results)
                if number_plate_coords:
                    x1, y1, x2, y2 = number_plate_coords
                    number_plate_img = frame[y1:y2, x1:x2]
                    cv2.imwrite(f'violations/{vehicle_number}_violation.jpg', number_plate_img)
                    print(f"Violation image saved for vehicle number {vehicle_number}")

    def process_video(self):
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter('output.avi', fourcc, 20.0, self.frame_size)

        while(self.cap.isOpened()):
            ret, frame = self.cap.read()
            if ret == True:
                frame = cv2.resize(frame, self.frame_size)
                original_frame = frame.copy()
                frame, results = object_detection(frame)

                if self.notice_generated:
                    # Convert frame to RGB for displaying in Tkinter
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    # Convert frame to ImageTk format
                    frame_tk = ImageTk.PhotoImage(image=Image.fromarray(frame_rgb))
                    # Display frame in Tkinter window
                    self.display_frame(frame_tk)

                rider_list = []
                head_list = []
                number_list = []

                for result in results:
                    x1,y1,x2,y2,cnf, clas = result
                    if clas == 0:
                        rider_list.append(result)
                    elif clas == 1:
                        head_list.append(result)
                    elif clas == 2:
                        number_list.append(result)

                for rdr in rider_list:
                    x1r, y1r, x2r, y2r, cnfr, clasr = rdr
                    for hd in head_list:
                        x1h, y1h, x2h, y2h, cnfh, clash = hd
                        if inside_box([x1r,y1r,x2r,y2r], [x1h,y1h,x2h,y2h]): 
                            try:
                                head_img = original_frame[y1h:y2h, x1h:x2h]
                                helmet_present = img_classify(head_img)
                            except:
                                helmet_present = [None]

                            if helmet_present[0] == False:
                                for num in number_list:
                                    x1_num, y1_num, x2_num, y2_num, conf_num, clas_num = num
                                    if inside_box([x1r,y1r,x2r,y2r], [x1_num, y1_num, x2_num, y2_num]):
                                        vehicle_number = extract_number_plate(original_frame, num)
                                        self.notice_generated = True
                                        self.generate_notice(vehicle_number)
                                        break
            else:
                break

            if self.save_video:
                out.write(frame)
            if self.save_img:
                cv2.imwrite('saved_frame.jpg', frame)
            if self.show_video:
                cv2.imshow('Frame', frame)

            key = cv2.waitKey(1)
            if key & 0xFF == ord('q'):
                break

        self.cap.release()
        cv2.destroyAllWindows()
        print('Execution completed')

    def display_frame(self, frame):
        if hasattr(self, 'panel'):
            self.panel.configure(image=frame)
            self.panel.image = frame
        else:
            self.panel = tk.Label(self.root, image=frame)
            self.panel.pack()

if __name__ == "__main__":
    app = App('video.mp4')
    app.root.mainloop()


Detected: head conf: 0.43  bbox: x1:436    y1:15    x2:536    y2:140
Detected: rider conf: 0.37  bbox: x1:414    y1:3    x2:576    y2:477
Detected: head conf: 0.62  bbox: x1:442    y1:311    x2:529    y2:414
Detected: rider conf: 0.56  bbox: x1:393    y1:301    x2:582    y2:477
Detected: head conf: 0.38  bbox: x1:397    y1:285    x2:574    y2:467
Detected: head conf: 0.65  bbox: x1:445    y1:200    x2:525    y2:306
Detected: rider conf: 0.59  bbox: x1:403    y1:191    x2:582    y2:478
Detected: rider conf: 0.66  bbox: x1:257    y1:334    x2:459    y2:480
Detected: head conf: 0.43  bbox: x1:307    y1:337    x2:385    y2:428
Detected: rider conf: 0.54  bbox: x1:235    y1:286    x2:460    y2:473
Detected: head conf: 0.4  bbox: x1:281    y1:303    x2:378    y2:389
Detected: rider conf: 0.56  bbox: x1:228    y1:261    x2:432    y2:479
Detected: rider conf: 0.46  bbox: x1:228    y1:233    x2:419    y2:469
Detected: rider conf: 0.37  bbox: x1:223    y1:183    x2:399    y2:475
Detected: rider 